<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_playgrounds_GGUF_4Bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gradio==4.22.0 transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=55

In [ ]:

#@title Built llama-cpp-python with BLAS or CUDA automatically, wait for few minutes .
import os,torch


if torch.cuda.is_available ():
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_CUBLAS=on"
else:
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

os.system (cmd)

0

In [6]:



from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
from llama_cpp.llama_chat_format import Llava15ChatHandler
import os,torch,json,shlex
from pathlib import Path
from PIL import Image
from huggingface_hub import hf_hub_download

IN_COLAB = False
Gbase="./generate/"
cache_dir="./hf/"

import torch
if torch.cuda.is_available():
    n_gpu_layers=-1
else :
    n_gpu_layers=0




description ="""<div style="font-family: Arial, sans-serif; padding: 20px;">

This showcases the use of llama-cpp-python to load various versions of 4-bit gemma 2B and 7B models, as well as a simple chat interface created with Gradio. It can be run on a mini computer or even on a phone, with satisfactory results. You are free to modify this code as you wish.
  <h2>🔗My social media links❤️</h2>

Follow <a href="https://www.facebook.com/braiml"  ytarget="_blank">🐍Brian's Page </a> if you want I share more tools .<br>
Follow<a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI</a>
 if you want more  videos .
<br>
  <a href="https://www.facebook.com/brian.pyai" target="_blank"> 📘facebook.com/brian.pyai</a>
 <br>
  <a href="https://www.facebook.com/braiml" target="_blank">🐍Brian's Page </a>
<br>

  <a href="https://www.facebook.com/lovelyai999" target="_blank">🥰AI Hot Shorts </a>



</div>

"""
import base64
import io
def image_to_base64_data_uri(file_path):
    i=Image.open(file_path)
    i.resize((256,256))
    img_byte_arr = io.BytesIO()
    i.save(img_byte_arr, format='PNG')
    base64_data = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{base64_data}"

modelsPath="./"

modelsPaths=["mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf",
"LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf",
"LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf",
"LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf",
"mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf",
"LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf",
"rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf",
"LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf",
"LoneStriker/zephyr-7b-gemma-sft-v0.1-GGUF/zephyr-7b-gemma-sft-v0.1-Q4_K_M.gguf",
"LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf"]


model_id="mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf" #@param ["mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","billborkowski/llava-NousResearch_Nous-Hermes-2-Vision-GGUF/NousResearch_Nous-Hermes-2-Vision-GGUF_Q4_0.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]

def downHgFile(url,targetDir=modelsPath):
    fileName=Path (model_id ).name
    repo =model_id[:-len(fileName)-1]
    print(fileName,repo)
    fileExists=os.path.exists(os.path.join(targetDir,fileName))

    print(repo,fileName,fileExists )
    if not fileExists:
        print ("Downloading file :")
        hf_hub_download(repo ,filename=fileName,local_dir=targetDir,local_dir_use_symlinks=False)
max_tokens=4096 # @param {type:"integer",min:10, max:8192}
n_ctx=4096 # @param {type:"integer",min:10, max:8192}
downHgFile(model_id )
#hf_hub_download("lovelyai999/temp" ,filename="mistral_7b_mmproj-v1_5_Q4_1.gguf",local_dir="./",local_dir_use_symlinks=False)
modelPath=os.path.join(modelsPath,Path(model_id).name)
if model_id=="mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf":
    if not os.path.exists("mistral_7b_mmproj-v1_5_Q4_1.gguf"):hf_hub_download("lovelyai999/temp" ,filename="mistral_7b_mmproj-v1_5_Q4_1.gguf",local_dir="./",local_dir_use_symlinks=False)
    chat_handler = Llava15ChatHandler(clip_model_path="mistral_7b_mmproj-v1_5_Q4_1.gguf",verbose=True)
    model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=4,max_tokens=4096,logits_all=True,n_ctx=n_ctx,chat_handler=chat_handler)
else:
    model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=4,max_tokens=4096,logits_all=True,n_ctx=n_ctx)
#draft_model=LlamaPromptLookupDecoding(num_pred_tokens=10) )

tokenizer = model.tokenize




import gradio as gr
import torch

from threading import Thread




partial_message = ""

def predict(message, history,top_k=300, top_p=0.95, temp=0.85, repeat_penalty=1.1,max_tokens=max_tokens):
    global partial_message
    try :messageT=message["text"]
    except :messageT=message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{messageT}
### Response :

"""
    print (message)
    #model_inputs = tokenizer(prompt.encode("utf-8"))
    #generate_kwargs = dict(top_k=top_k, top_p=top_p, temp=temp, repeat_penalty=repeat_penalty)
    generate_kwargs=dict (suffix=None, max_tokens=max_tokens, temperature=temp, top_p=top_p, min_p=0.05, typical_p=1.0, logprobs=None, echo=False, stop=[], frequency_penalty=0.0, presence_penalty=0.0, repeat_penalty=1.1, top_k=top_k, stream=True , seed=None, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None, logit_bias=None)


    partial_message = ""
    if isinstance(message,dict) and message["files"] and message["files"][0]["path"] and message["files"][0]['mime_type'].startswith("image"):
        messages = [
        {"role": "system", "content": "You are an assistant who perfectly describes images."},
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url":message["files"][0]["path"]}},
                {"type" : "text", "text": messageT}
            ]
        }]
        outputs =model.create_completion(prompt =json.dumps (messages) ,**generate_kwargs)
    else:
        outputs =model.create_completion(prompt =prompt ,**generate_kwargs)


    for chunk in outputs:
        #print (chunk )
        content = chunk["choices"][0]["text"]
        #print(content)
        if content:
            partial_message+=content
            yield partial_message
    """
    for new_token in model.generate(model_inputs,**generate_kwargs):
            #print (new_token)
            if new_token in (213,7221,1):break
            s=model.detokenize([new_token])
            #print (s)
            if isinstance(s,bytes):
                try:partial_message += s.decode("utf-8")
                except :break
            yield partial_message
        """

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)

user="brian" #@param {type:"string"}
password="pwd" #@param {type:"string"}
auth=(user,password)

gr.ChatInterface(predict,chatbot=chatbot,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1,description=description,
additional_inputs=[gr.Number(value=100,minimum =0,maximum =1000,precision=0,show_label=True ,label="Topic K"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Topic P"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Temperature"),gr.Number(value=1.1,minimum =1,maximum =3,precision=3,show_label=True ,label="Repeat penalty") ,gr.Number(value=4096,minimum =0,maximum =8192,precision=0,show_label=True ,label="max_tokens") ],multimodal=True ).launch(debug=True,share=True ,inline=False,auth=auth )

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./llava-v1.6-mistral-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32     

llava-v1.6-mistral-7b.Q4_K_M.gguf mradermacher/llava-v1.6-mistral-7b-GGUF
mradermacher/llava-v1.6-mistral-7b-GGUF llava-v1.6-mistral-7b.Q4_K_M.gguf True


llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22:                    tokenizer.chat_template str              = {{ bos_token }}{% for message in mess...
llama_model_loader: - kv  23: 

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://449a09ab98727f31ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
{'text': 'Describe the image .', 'files': [{'path': '/tmp/gradio/09368574af1e2802967cf8844cd49ab97bfdb247/IMG20230701140318.jpg', 'url': '/file=/tmp/gradio/09368574af1e2802967cf8844cd49ab97bfdb247/IMG20230701140318.jpg', 'size': 3264871, 'orig_name': 'IMG20230701140318.jpg', 'mime_type': 'image/jpeg', 'is_stream': False, 'meta': {'_type': 'gradio.FileData'}}]}



llama_print_timings:        load time =   36540.55 ms
llama_print_timings:      sample time =      67.30 ms /   110 runs   (    0.61 ms per token,  1634.52 tokens per second)
llama_print_timings: prompt eval time =   36540.29 ms /   126 tokens (  290.00 ms per token,     3.45 tokens per second)
llama_print_timings:        eval time =   22027.42 ms /   109 runs   (  202.09 ms per token,     4.95 tokens per second)
llama_print_timings:       total time =   59677.33 ms /   235 tokens


...

{'text': 'Describe the photo .', 'files': [{'path': '/tmp/gradio/90655eb8cee609248aa324f99580a2a76d550ef5/d2.jpg', 'url': '/file=/tmp/gradio/90655eb8cee609248aa324f99580a2a76d550ef5/d2.jpg', 'size': 24706, 'orig_name': 'd2.jpg', 'mime_type': 'image/jpeg', 'is_stream': False, 'meta': {'_type': 'gradio.FileData'}}]}


Llama.generate: prefix-match hit

llama_print_timings:        load time =   36540.55 ms
llama_print_timings:      sample time =       1.16 ms /     2 runs   (    0.58 ms per token,  1730.10 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     416.32 ms /     2 runs   (  208.16 ms per token,     4.80 tokens per second)
llama_print_timings:       total time =     428.46 ms /     3 tokens


{'text': 'Describe the image .', 'files': [{'path': '/tmp/gradio/90bcf2e8e8625c6938c138055f9274a32e2d511b/2024012305400411.jpg', 'url': '/file=/tmp/gradio/90bcf2e8e8625c6938c138055f9274a32e2d511b/2024012305400411.jpg', 'size': 1854466, 'orig_name': '2024012305400411.jpg', 'mime_type': 'image/jpeg', 'is_stream': False, 'meta': {'_type': 'gradio.FileData'}}]}


Llama.generate: prefix-match hit

llama_print_timings:        load time =   36540.55 ms
llama_print_timings:      sample time =     105.83 ms /   173 runs   (    0.61 ms per token,  1634.71 tokens per second)
llama_print_timings: prompt eval time =   33002.53 ms /    75 tokens (  440.03 ms per token,     2.27 tokens per second)
llama_print_timings:        eval time =   36766.00 ms /   172 runs   (  213.76 ms per token,     4.68 tokens per second)
llama_print_timings:       total time =   71311.27 ms /   247 tokens


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://449a09ab98727f31ff.gradio.live


In [ ]:
!ls -a